# This Notebook Downloads and Processes Historical MET Data from NASA Power Platform via API

In [71]:
import pandas as pd
import numpy as np
import requests
import os
from datetime import datetime, timezone, timedelta

In [103]:
# Load Stations list
stations_df = pd.read_csv('Cleaned Stations List.csv')
stations_df.head()

,LOCATION (STATION),LOCATION DESCRIPTION,LATITUDE,LONGITUDE,ALTITUDE(M),STATE
0,ILORIN (MET STATION),HEAD QUARTERS,8 30 26.9 N,4 35 50.85 E,364.0,KWARA
1,ALAPA (MET STATION),"GOV. SEC. SCH, ALAPA",8 37 28.98 N,4 23 21.48 E,379.0,KWARA
2,MOSHE GADA (WATER LOGGER),MOSHE GADA (BRIDGE),9 12 36.5 N,3 51 57.6 E,249.0,KWARA
3,KAIAMA (MET STATION),"GOV. SEC. SCH, KAIAMA",9 34 55.4 N,3 55 43.3 E,NaN,KWARA
4,OKUTA (MET STATION),"OKUTA PROJECT OFFICE, LNRBDA",9 13 39.7 N,3 11 43.6 E,381.0,KWARA


In [73]:
# Create Station Type

stations_df['STATION TYPE'] = stations_df['LOCATION (STATION)'].str.extract(r'\((.*?)\)')

In [74]:
# Update station location 
stations_df['LOCATION (STATION)'] = stations_df['LOCATION (STATION)'].str.extract(r'^(.*?)\s*\(')
stations_df.head()

,LOCATION (STATION),LOCATION DESCRIPTION,LATITUDE,LONGITUDE,ALTITUDE(M),STATE,STATION TYPE
0,ILORIN,HEAD QUARTERS,8 30 26.9 N,4 35 50.85 E,364.0,KWARA,MET STATION
1,ALAPA,"GOV. SEC. SCH, ALAPA",8 37 28.98 N,4 23 21.48 E,379.0,KWARA,MET STATION
2,MOSHE GADA,MOSHE GADA (BRIDGE),9 12 36.5 N,3 51 57.6 E,249.0,KWARA,WATER LOGGER
3,KAIAMA,"GOV. SEC. SCH, KAIAMA",9 34 55.4 N,3 55 43.3 E,NaN,KWARA,MET STATION
4,OKUTA,"OKUTA PROJECT OFFICE, LNRBDA",9 13 39.7 N,3 11 43.6 E,381.0,KWARA,MET STATION


In [102]:
stations_df = stations_df[['LOCATION (STATION)', 'STATION TYPE',  'LATITUDE', 'LONGITUDE', 'ALTITUDE(M)', 'STATE', 'LOCATION DESCRIPTION']]
stations_df.head()

,LOCATION (STATION),STATION TYPE,LATITUDE,LONGITUDE,ALTITUDE(M),STATE,LOCATION DESCRIPTION
0,ILORIN,MET STATION,8.5075,4.5975,364.0,KWARA,HEAD QUARTERS
1,ALAPA,MET STATION,8.6247,4.3893,379.0,KWARA,"GOV. SEC. SCH, ALAPA"
2,MOSHE GADA,WATER LOGGER,9.2101,3.8660,249.0,KWARA,MOSHE GADA (BRIDGE)
3,KAIAMA,MET STATION,9.5821,3.9287,NaN,KWARA,"GOV. SEC. SCH, KAIAMA"
4,OKUTA,MET STATION,9.2277,3.1954,381.0,KWARA,"OKUTA PROJECT OFFICE, LNRBDA"


In [76]:
# Convert the Coordinates to decimals:
def dms_to_decimal(dms_str):
    # Split DMS string into components
    parts = dms_str.split()
    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2].replace("''", ""))
    direction = parts[3]

    # Calculate decimal degrees
    decimal = degrees + (minutes / 60) + (seconds / 3600)

    # Negate if direction is South or West
    if direction in ['S', 'W']:
        decimal = -decimal

    return decimal

# Example usage
dms_coordinate = "8 08 12.6 N" #"8 30 26.9 N"
latitude_decimal = dms_to_decimal(dms_coordinate)

print(f"Decimal Latitude: {latitude_decimal}")

Decimal Latitude: 8.136833333333334


In [77]:
stations_df['LATITUDE'] = stations_df['LATITUDE'].apply(dms_to_decimal)
stations_df['LONGITUDE'] = stations_df['LONGITUDE'].apply(dms_to_decimal)

In [101]:
stations_df.head()

,LOCATION (STATION),STATION TYPE,LATITUDE,LONGITUDE,ALTITUDE(M),STATE,LOCATION DESCRIPTION
0,ILORIN,MET STATION,8.5075,4.5975,364.0,KWARA,HEAD QUARTERS
1,ALAPA,MET STATION,8.6247,4.3893,379.0,KWARA,"GOV. SEC. SCH, ALAPA"
2,MOSHE GADA,WATER LOGGER,9.2101,3.8660,249.0,KWARA,MOSHE GADA (BRIDGE)
3,KAIAMA,MET STATION,9.5821,3.9287,NaN,KWARA,"GOV. SEC. SCH, KAIAMA"
4,OKUTA,MET STATION,9.2277,3.1954,381.0,KWARA,"OKUTA PROJECT OFFICE, LNRBDA"


In [79]:
# Bounding Box Coordinates

lat_min, lat_max  = stations_df['LATITUDE'].min(), stations_df['LATITUDE'].max()
lon_min, lon_max = stations_df['LONGITUDE'].min(), stations_df['LONGITUDE'].max()

print('lat_min: ', lat_min,)
print('lat_max: ', lat_max,)
print('lon_min: ', lon_min,)
print('lon_max: ', lon_max,)

lat_min:  7.600427777777777
lat_max:  9.582055555555556
lon_min:  3.195444444444444
lon_max:  6.740333333333333


In [80]:
# List of Parameters

parameters_list = [
    'T2M_RANGE', 'QV2M', 'RH2M', 'PRECTOTCORR', 'ALLSKY_SFC_SW_DWN', 
    'WS2M_MAX', 'WS2M_MIN', 'T2M_MAX', 'T2M_MIN'
]

# Format parameters
parameters_formatted = ""
for param in parameters_list:
    parameters_formatted += f'{param},'
parameters_formatted = parameters_formatted[:-1]
    
        
print(parameters_formatted)

username = 'DeBoss'

# Start and end dates in format YYYYMMDD
start_date = '19810101'
end_date = '20241031'

# Time format UTC or LST
timeformat = 'utc'

T2M_RANGE,QV2M,RH2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,T2M_MAX,T2M_MIN


In [15]:
coordinate_pairs = zip(stations_df['LATITUDE'], stations_df['LONGITUDE'])

# Create Download folder
download_dir = 'Met Downloads'
if not os.path.exists('Met Downloads'):
    os.mkdir(download_dir)
    
for coordinate in coordinate_pairs:
    latitude, longitude = coordinate[0], coordinate[1]
    if not f'{round(latitude, 4)}_{round(longitude, 4)}.csv' in download_dir:
        url = f"https://power.larc.nasa.gov/api/temporal/daily/point?start={start_date}&end={end_date}&latitude={latitude}&longitude={longitude}&community=ag&parameters={parameters_formatted}&format=csv&user={username}&header=false&time-standard={timeformat}"
        data = requests.get(url)
        with open(f'{download_dir}/{round(latitude, 4)}_{round(longitude, 4)}.csv', mode='wb') as file:
            file.write(data.content)

In [81]:
downloads_dir = os.listdir(download_dir)

# Load the downloaded files:
files = [file for file in downloads_dir if file.endswith('.csv')]
dfs = []
for file in files:
    df = pd.read_csv(os.path.join(download_dir, file), na_values=-999)
    df['LAT'], df['LON'] = file.replace('.csv', '').split('_')
    dfs.append(df)

# Concatenate the data for processing
met_df = pd.concat(dfs)
met_df

,YEAR,DOY,T2M_RANGE,QV2M,RH2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,T2M_MAX,T2M_MIN,LAT,LON
0,1981,1,11.41,13.24,67.31,0.00,NaN,3.57,0.41,31.00,19.58,7.6004,6.238
1,1981,2,12.50,13.37,70.00,0.01,NaN,2.25,0.52,30.85,18.35,7.6004,6.238
2,1981,3,10.18,12.76,69.88,0.03,NaN,2.52,0.47,29.27,19.09,7.6004,6.238
3,1981,4,10.34,10.19,59.81,0.00,NaN,2.18,0.29,28.24,17.91,7.6004,6.238
4,1981,5,12.40,10.38,62.00,0.00,NaN,2.84,0.71,28.92,16.52,7.6004,6.238
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16005,2024,301,12.24,14.77,66.12,0.00,NaN,2.48,0.73,34.29,22.05,9.5821,3.9287
16006,2024,302,12.55,15.08,68.19,0.03,NaN,2.87,0.84,33.69,21.15,9.5821,3.9287
16007,2024,303,13.65,13.67,61.75,0.00,NaN,2.98,0.21,35.01,21.37,9.5821,3.9287
16008,2024,304,14.87,14.16,63.62,0.01,NaN,2.96,0.45,35.67,20.80,9.5821,3.9287


In [82]:
files

['7.6004_6.238.csv',
 '7.828_5.8626.csv',
 '8.0824_5.245.csv',
 '8.1076_5.5604.csv',
 '8.1368_5.5173.csv',
 '8.1375_5.3298.csv',
 '8.1439_5.1081.csv',
 '8.2122_5.6359.csv',
 '8.2474_4.8141.csv',
 '8.2519_6.7403.csv',
 '8.285_5.6689.csv',
 '8.2927_5.6541.csv',
 '8.5075_4.5975.csv',
 '8.6164_4.767.csv',
 '8.6247_4.3893.csv',
 '8.7239_5.7595.csv',
 '8.836_4.8441.csv',
 '8.8617_5.4276.csv',
 '8.9302_5.4499.csv',
 '9.0048_5.1502.csv',
 '9.1931_3.2138.csv',
 '9.2101_3.866.csv',
 '9.2277_3.1954.csv',
 '9.5821_3.9287.csv']

## Data Processing (Daily Timeframe)

In [83]:
met_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384240 entries, 0 to 16009
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   YEAR               384240 non-null  int64  
 1   DOY                384240 non-null  int64  
 2   T2M_RANGE          384168 non-null  float64
 3   QV2M               384168 non-null  float64
 4   RH2M               384168 non-null  float64
 5   PRECTOTCORR        384168 non-null  float64
 6   ALLSKY_SFC_SW_DWN  355032 non-null  float64
 7   WS2M_MAX           384168 non-null  float64
 8   WS2M_MIN           384168 non-null  float64
 9   T2M_MAX            384168 non-null  float64
 10  T2M_MIN            384168 non-null  float64
 11  LAT                384240 non-null  object 
 12  LON                384240 non-null  object 
dtypes: float64(9), int64(2), object(2)
memory usage: 41.0+ MB


In [84]:
def apply_time_info(year, day_of_year):
    date = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)
    
    #Time is in UTC+0
    date_with_zero_time = date.replace(hour=0, minute=0, second=0, microsecond=0, tzinfo=timezone(timedelta(hours=0)))
    return date_with_zero_time

# Apply function
met_df['TIME'] = df.apply(lambda row: apply_time_info(row['YEAR'], row['DOY']), axis=1)
met_df.head()

,YEAR,DOY,T2M_RANGE,QV2M,RH2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,T2M_MAX,T2M_MIN,LAT,LON,TIME
0,1981,1,11.41,13.24,67.31,0.00,NaN,3.57,0.41,31.00,19.58,7.6004,6.238,1981-01-01 00:00:00+00:00
1,1981,2,12.50,13.37,70.00,0.01,NaN,2.25,0.52,30.85,18.35,7.6004,6.238,1981-01-02 00:00:00+00:00
2,1981,3,10.18,12.76,69.88,0.03,NaN,2.52,0.47,29.27,19.09,7.6004,6.238,1981-01-03 00:00:00+00:00
3,1981,4,10.34,10.19,59.81,0.00,NaN,2.18,0.29,28.24,17.91,7.6004,6.238,1981-01-04 00:00:00+00:00
4,1981,5,12.40,10.38,62.00,0.00,NaN,2.84,0.71,28.92,16.52,7.6004,6.238,1981-01-05 00:00:00+00:00


In [85]:
# Drop redundant columns
met_df.drop(columns=['YEAR', 'DOY'], inplace=True)
met_df.head()

,T2M_RANGE,QV2M,RH2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,T2M_MAX,T2M_MIN,LAT,LON,TIME
0,11.41,13.24,67.31,0.00,NaN,3.57,0.41,31.00,19.58,7.6004,6.238,1981-01-01 00:00:00+00:00
1,12.50,13.37,70.00,0.01,NaN,2.25,0.52,30.85,18.35,7.6004,6.238,1981-01-02 00:00:00+00:00
2,10.18,12.76,69.88,0.03,NaN,2.52,0.47,29.27,19.09,7.6004,6.238,1981-01-03 00:00:00+00:00
3,10.34,10.19,59.81,0.00,NaN,2.18,0.29,28.24,17.91,7.6004,6.238,1981-01-04 00:00:00+00:00
4,12.40,10.38,62.00,0.00,NaN,2.84,0.71,28.92,16.52,7.6004,6.238,1981-01-05 00:00:00+00:00


In [86]:
# Change datatypes:
met_df = met_df.astype({'LAT': 'float64', 'LON': 'float64'})

# Daily averages of Temperature, Windspeed, etc

met_df['WS2M_AVG'] = met_df[['WS2M_MIN', 'WS2M_MAX']].mean(axis=1)  # AVG Windspeed
met_df['T2M_AVG'] = met_df[['T2M_MIN', 'T2M_MAX']].mean(axis=1)     # AVG Temperature

met_df.head()

,T2M_RANGE,QV2M,RH2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,T2M_MAX,T2M_MIN,LAT,LON,TIME,WS2M_AVG,T2M_AVG
0,11.41,13.24,67.31,0.00,NaN,3.57,0.41,31.00,19.58,7.6004,6.238,1981-01-01 00:00:00+00:00,1.990,25.290
1,12.50,13.37,70.00,0.01,NaN,2.25,0.52,30.85,18.35,7.6004,6.238,1981-01-02 00:00:00+00:00,1.385,24.600
2,10.18,12.76,69.88,0.03,NaN,2.52,0.47,29.27,19.09,7.6004,6.238,1981-01-03 00:00:00+00:00,1.495,24.180
3,10.34,10.19,59.81,0.00,NaN,2.18,0.29,28.24,17.91,7.6004,6.238,1981-01-04 00:00:00+00:00,1.235,23.075
4,12.40,10.38,62.00,0.00,NaN,2.84,0.71,28.92,16.52,7.6004,6.238,1981-01-05 00:00:00+00:00,1.775,22.720


In [87]:
met_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384240 entries, 0 to 16009
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   T2M_RANGE          384168 non-null  float64            
 1   QV2M               384168 non-null  float64            
 2   RH2M               384168 non-null  float64            
 3   PRECTOTCORR        384168 non-null  float64            
 4   ALLSKY_SFC_SW_DWN  355032 non-null  float64            
 5   WS2M_MAX           384168 non-null  float64            
 6   WS2M_MIN           384168 non-null  float64            
 7   T2M_MAX            384168 non-null  float64            
 8   T2M_MIN            384168 non-null  float64            
 9   LAT                384240 non-null  float64            
 10  LON                384240 non-null  float64            
 11  TIME               384240 non-null  datetime64[ns, UTC]
 12  WS2M_AVG           384168 non-n

In [88]:
# Pull other location infos to this dataframe
stations_df[['LATITUDE', 'LONGITUDE']] = round(stations_df[['LATITUDE', 'LONGITUDE']], 4)

met_df = pd.merge(met_df, stations_df, left_on=['LAT', 'LON'], right_on=['LATITUDE', 'LONGITUDE'], how='left')
met_df.head()

,T2M_RANGE,QV2M,RH2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,T2M_MAX,T2M_MIN,LAT,...,TIME,WS2M_AVG,T2M_AVG,LOCATION (STATION),STATION TYPE,LATITUDE,LONGITUDE,ALTITUDE(M),STATE,LOCATION DESCRIPTION
0,11.41,13.24,67.31,0.00,NaN,3.57,0.41,31.00,19.58,7.6004,...,1981-01-01 00:00:00+00:00,1.990,25.290,OGAMINANA,MET STATION,7.6004,6.238,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
1,12.50,13.37,70.00,0.01,NaN,2.25,0.52,30.85,18.35,7.6004,...,1981-01-02 00:00:00+00:00,1.385,24.600,OGAMINANA,MET STATION,7.6004,6.238,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
2,10.18,12.76,69.88,0.03,NaN,2.52,0.47,29.27,19.09,7.6004,...,1981-01-03 00:00:00+00:00,1.495,24.180,OGAMINANA,MET STATION,7.6004,6.238,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
3,10.34,10.19,59.81,0.00,NaN,2.18,0.29,28.24,17.91,7.6004,...,1981-01-04 00:00:00+00:00,1.235,23.075,OGAMINANA,MET STATION,7.6004,6.238,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
4,12.40,10.38,62.00,0.00,NaN,2.84,0.71,28.92,16.52,7.6004,...,1981-01-05 00:00:00+00:00,1.775,22.720,OGAMINANA,MET STATION,7.6004,6.238,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"


In [89]:
# Drop redundant columns
met_df.columns

Index(['T2M_RANGE', 'QV2M', 'RH2M', 'PRECTOTCORR', 'ALLSKY_SFC_SW_DWN',
       'WS2M_MAX', 'WS2M_MIN', 'T2M_MAX', 'T2M_MIN', 'LAT', 'LON', 'TIME',
       'WS2M_AVG', 'T2M_AVG', 'LOCATION (STATION)', 'STATION TYPE', 'LATITUDE',
       'LONGITUDE', 'ALTITUDE(M)', 'STATE', 'LOCATION DESCRIPTION'],
      dtype='object')

In [90]:
irrelevant_cols = ['LATITUDE', 'LONGITUDE']
met_df.drop(columns=irrelevant_cols, inplace=True)

In [93]:
# Output folder
if not os.path.exists('output'):
    output_folder = os.mkdir('output')

In [94]:
# Export Daily Data to CSV
met_df.to_csv('output/Met_Daily_All_Stations_19810101_20241031.csv', index=False)

## Resample (Monthly Timeframe)

In [95]:
# Copy Dataframe
met_df_monthly = met_df.copy()

In [96]:
# Set Time as Index Column
met_df_monthly.set_index('TIME', inplace=True)
met_df_monthly.head()

,T2M_RANGE,QV2M,RH2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,T2M_MAX,T2M_MIN,LAT,LON,WS2M_AVG,T2M_AVG,LOCATION (STATION),STATION TYPE,ALTITUDE(M),STATE,LOCATION DESCRIPTION
TIME,,,,,,,,,,,,,,,,,,
1981-01-01 00:00:00+00:00,11.41,13.24,67.31,0.00,NaN,3.57,0.41,31.00,19.58,7.6004,6.238,1.990,25.290,OGAMINANA,MET STATION,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
1981-01-02 00:00:00+00:00,12.50,13.37,70.00,0.01,NaN,2.25,0.52,30.85,18.35,7.6004,6.238,1.385,24.600,OGAMINANA,MET STATION,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
1981-01-03 00:00:00+00:00,10.18,12.76,69.88,0.03,NaN,2.52,0.47,29.27,19.09,7.6004,6.238,1.495,24.180,OGAMINANA,MET STATION,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
1981-01-04 00:00:00+00:00,10.34,10.19,59.81,0.00,NaN,2.18,0.29,28.24,17.91,7.6004,6.238,1.235,23.075,OGAMINANA,MET STATION,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
1981-01-05 00:00:00+00:00,12.40,10.38,62.00,0.00,NaN,2.84,0.71,28.92,16.52,7.6004,6.238,1.775,22.720,OGAMINANA,MET STATION,NaN,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"


In [97]:
# Group by LAT and LON and resample to monthly with named aggregation
dimensions_cols = ['LAT', 'LON', 'LOCATION (STATION)', 'STATION TYPE', 'STATE', 'LOCATION DESCRIPTION']
met_df_monthly = met_df_monthly.groupby(dimensions_cols).resample('M').agg(
    T2M_RANGE = pd.NamedAgg('T2M_RANGE', 'mean'),
    QV2M               = pd.NamedAgg('QV2M', 'mean'),
    RH2M               = pd.NamedAgg('RH2M', 'mean'),
    PRECTOTCORR        = pd.NamedAgg('PRECTOTCORR', 'sum'),
    PRECTOTCORR_AVG    = pd.NamedAgg('PRECTOTCORR', 'mean'),
    ALLSKY_SFC_SW_DWN  = pd.NamedAgg('ALLSKY_SFC_SW_DWN', 'mean'),
    WS2M_MAX           = pd.NamedAgg('WS2M_MAX', 'max'),
    WS2M_MIN           = pd.NamedAgg('WS2M_MIN', 'min'),
    WS2M_AVG           = pd.NamedAgg('WS2M_AVG', 'mean'),
    T2M_MAX            = pd.NamedAgg('T2M_MAX', 'max'),
    T2M_MIN            = pd.NamedAgg('T2M_MIN', 'min'),
    T2M_AVG            = pd.NamedAgg('T2M_AVG', 'mean'),
).reset_index() 

met_df_monthly

,LAT,LON,LOCATION (STATION),STATION TYPE,STATE,LOCATION DESCRIPTION,TIME,T2M_RANGE,QV2M,RH2M,PRECTOTCORR,PRECTOTCORR_AVG,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,WS2M_AVG,T2M_MAX,T2M_MIN,T2M_AVG
0,7.6004,6.2380,OGAMINANA,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA",1981-01-31 00:00:00+00:00,11.327742,11.647097,63.495484,12.21,0.393871,NaN,4.33,0.06,1.394032,31.33,15.71,24.176774
1,7.6004,6.2380,OGAMINANA,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA",1981-02-28 00:00:00+00:00,12.318929,13.059286,63.747143,3.99,0.142500,NaN,5.27,0.05,1.987857,34.12,15.02,26.059643
2,7.6004,6.2380,OGAMINANA,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA",1981-03-31 00:00:00+00:00,9.952903,16.107419,74.282581,34.34,1.107742,NaN,4.82,0.02,2.556452,34.92,19.69,27.294355
3,7.6004,6.2380,OGAMINANA,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA",1981-04-30 00:00:00+00:00,8.674333,17.612667,80.401667,197.03,6.567667,NaN,4.29,0.24,2.287833,35.30,22.04,27.289333
4,7.6004,6.2380,OGAMINANA,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA",1981-05-31 00:00:00+00:00,6.847419,17.643548,86.177419,143.71,4.635806,NaN,3.82,0.04,1.873065,30.78,20.96,25.672903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12619,9.5821,3.9287,KAIAMA,MET STATION,KWARA,"GOV. SEC. SCH, KAIAMA",2024-06-30 00:00:00+00:00,8.260333,18.042000,79.058000,263.32,8.777333,16.949333,4.38,0.19,1.995167,34.67,22.21,27.631167
12620,9.5821,3.9287,KAIAMA,MET STATION,KWARA,"GOV. SEC. SCH, KAIAMA",2024-07-31 00:00:00+00:00,8.230357,17.026429,78.370357,159.88,5.710000,19.110000,4.91,0.19,2.294821,35.47,21.87,26.872857
12621,9.5821,3.9287,KAIAMA,MET STATION,KWARA,"GOV. SEC. SCH, KAIAMA",2024-08-31 00:00:00+00:00,7.713226,17.221935,82.232903,193.21,6.232581,NaN,5.51,0.39,2.574839,32.69,21.36,26.121452
12622,9.5821,3.9287,KAIAMA,MET STATION,KWARA,"GOV. SEC. SCH, KAIAMA",2024-09-30 00:00:00+00:00,7.228000,17.590667,84.658000,297.80,9.926667,NaN,4.77,0.08,1.831500,31.93,21.08,25.794500


In [98]:
met_df_monthly.columns

Index(['LAT', 'LON', 'LOCATION (STATION)', 'STATION TYPE', 'STATE',
       'LOCATION DESCRIPTION', 'TIME', 'T2M_RANGE', 'QV2M', 'RH2M',
       'PRECTOTCORR', 'PRECTOTCORR_AVG', 'ALLSKY_SFC_SW_DWN', 'WS2M_MAX',
       'WS2M_MIN', 'WS2M_AVG', 'T2M_MAX', 'T2M_MIN', 'T2M_AVG'],
      dtype='object')

In [99]:
# Reorder the columns

met_df_monthly = met_df_monthly[['TIME', 'LAT', 'LON', 'LOCATION (STATION)', 
                                 'T2M_RANGE', 'QV2M', 'RH2M', 'PRECTOTCORR', 
                                 'PRECTOTCORR_AVG', 'ALLSKY_SFC_SW_DWN', 'WS2M_MAX',
                                 'WS2M_MIN', 'WS2M_AVG', 'T2M_MAX', 'T2M_MIN', 'T2M_AVG',
                                 'STATION TYPE', 'STATE', 'LOCATION DESCRIPTION'
                                ]]
met_df_monthly.head()

,TIME,LAT,LON,LOCATION (STATION),T2M_RANGE,QV2M,RH2M,PRECTOTCORR,PRECTOTCORR_AVG,ALLSKY_SFC_SW_DWN,WS2M_MAX,WS2M_MIN,WS2M_AVG,T2M_MAX,T2M_MIN,T2M_AVG,STATION TYPE,STATE,LOCATION DESCRIPTION
0,1981-01-31 00:00:00+00:00,7.6004,6.238,OGAMINANA,11.327742,11.647097,63.495484,12.21,0.393871,NaN,4.33,0.06,1.394032,31.33,15.71,24.176774,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
1,1981-02-28 00:00:00+00:00,7.6004,6.238,OGAMINANA,12.318929,13.059286,63.747143,3.99,0.142500,NaN,5.27,0.05,1.987857,34.12,15.02,26.059643,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
2,1981-03-31 00:00:00+00:00,7.6004,6.238,OGAMINANA,9.952903,16.107419,74.282581,34.34,1.107742,NaN,4.82,0.02,2.556452,34.92,19.69,27.294355,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
3,1981-04-30 00:00:00+00:00,7.6004,6.238,OGAMINANA,8.674333,17.612667,80.401667,197.03,6.567667,NaN,4.29,0.24,2.287833,35.30,22.04,27.289333,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"
4,1981-05-31 00:00:00+00:00,7.6004,6.238,OGAMINANA,6.847419,17.643548,86.177419,143.71,4.635806,NaN,3.82,0.04,1.873065,30.78,20.96,25.672903,MET STATION,KOGI,"EBIRA COMM SEC SCH, OGAMINANA"


In [100]:
# Export to CSV file
met_df_monthly.to_csv('output/Met_Monthly_All_Stations_19810101_20241031.csv', index=False)